In [1]:
data_var = '2024-02-02'

In [2]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [3]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [4]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [5]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [6]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [7]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
12208,2024-02-02,Eua Nba,00:00,Utah Jazz,Philadelphia 76ers,1.50,2.72,238.5,1.83,2.07,-5.5,2.01,1.93,lfKU00uR,0.666667,0.367647,0.546448,0.483092,0.034314,0.6,0.4,NaN,NaN,244.982,100.913292,0.411921,200.284,67.338380,0.336214,262.908,194.692,312.18,252.52,0.0,0.0,0.0,0.0,0.408848,0.087029,0.028715,2.54,0.508,0.984252,0.480277,0.8,0.319723,-3.78,-0.756,-2.275132,0.549081,0.3,-0.249081
12209,2024-02-02,Eua Nba,21:00,Detroit Pistons,Los Angeles Clippers,4.04,1.18,233.5,1.81,1.99,9.5,2.08,1.75,6B01ev3l,0.247525,0.847458,0.552486,0.502513,0.094982,0.2,0.4,NaN,NaN,569.554,344.467907,0.604803,210.042,99.616107,0.474268,569.554,215.120,814.80,141.25,1.0,0.0,0.0,0.0,0.774837,0.066989,0.121851,3.54,0.708,4.293785,0.310702,0.3,-0.010702,0.81,0.162,1.111111,0.593875,0.6,0.006125
12210,2024-02-02,Eua Nba,21:00,Washington Wizards,Miami Heat,3.47,1.24,232.5,1.84,1.96,7.5,1.98,1.82,0Ya5fbIf,0.288184,0.806452,0.543478,0.510204,0.094636,0.2,0.2,NaN,NaN,460.014,220.237511,0.478763,242.884,109.265738,0.449868,530.134,280.034,729.21,312.83,1.0,0.0,0.0,0.0,0.669575,0.044659,0.059546,-5.00,-1.000,-2.470000,0.290305,0.1,-0.190305,0.05,0.010,24.000000,0.412449,0.6,0.187551
12211,2024-02-02,Eua Nba,21:30,Atlanta Hawks,Phoenix Suns,2.16,1.59,247.5,1.82,1.99,2.5,2.05,1.77,vPb9gIX0,0.462963,0.628931,0.549451,0.502513,0.091894,0.4,0.4,NaN,NaN,283.442,90.702715,0.320004,207.256,46.212824,0.222975,199.708,229.256,211.14,216.24,0.0,0.0,0.0,0.0,0.214960,0.063101,0.103660,-0.44,-0.088,-13.181818,0.608723,0.5,-0.108723,1.23,0.246,2.398374,0.536573,0.6,0.063427
12212,2024-02-02,Eua Nba,22:00,Houston Rockets,Toronto Raptors,1.39,2.69,227.5,1.85,1.95,-5.5,2.01,1.93,MTTa8hdS,0.719424,0.371747,0.540541,0.512821,0.091172,0.4,0.8,NaN,NaN,272.196,134.026573,0.492390,291.824,100.137101,0.343142,283.600,398.754,224.73,360.00,0.0,0.0,0.0,0.0,0.450607,0.037216,0.028715,-0.47,-0.094,-4.148936,0.533362,0.4,-0.133362,-2.85,-0.570,-2.964912,0.405056,0.3,-0.105056
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12341,2024-02-02,Arábia Saudita Premier League,10:00,Dbae,Al Wehda,9.85,1.04,170.5,1.85,1.85,23.5,1.97,1.74,WxMofBmJ,0.101523,0.961538,0.540541,0.540541,0.063061,0.0,0.4,NaN,NaN,NaN,NaN,NaN,176.690,83.635550,0.473346,0.000,193.142,785.40,241.87,1.0,0.0,0.0,0.0,1.144097,0.000000,0.087674,0.00,0.000,inf,0.000000,0.0,0.000000,-2.68,-0.536,-0.074627,0.000000,0.0,0.000000
12342,2024-02-02,Arábia Saudita Premier League,11:00,Al Nasr,Al Khaleej,1.04,9.85,167.5,1.83,1.87,-26.5,2.07,1.67,2Fu5a6Pb,0.961538,0.101523,0.546448,0.534759,0.063061,0.2,0.6,NaN,NaN,574.260,1054.207493,1.835767,408.254,297.605947,0.728973,106.610,308.724,97.02,425.79,0.0,0.0,0.0,0.0,1.144097,0.015289,0.151253,-0.18,-0.036,-1.111111,0.000000,0.0,0.000000,-1.71,-0.342,-25.877193,0.000000,0.0,0.000000
12343,2024-02-02,Chile Lnb,20:30,U. De Concepcion,CD Puerto Varas,1.08,6.95,159.5,1.87,1.79,-15.5,2.06,1.68,SbpwISsb,0.925926,0.143885,0.534759,0.558659,0.069811,0.0,0.0,NaN,NaN,NaN,NaN,NaN,155.076,88.375824,0.569887,0.000,0.000,0.00,88.44,0.0,0.0,0.0,0.0,1.033802,0.030912,0.143690,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
12344,2024-02-02,Europa Ewbl Feminina,14:00,Polonia W. F,Frankivsk Halichina F,1.12,5.66,146.5,1.83,1.85,-26.5,2.07,1.67,dUK1e1j

## Apostas do dia

### Over v0 (LDA)

In [8]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,00:00,Eua Nba,Utah Jazz,Philadelphia 76ers,238.5,1.83,1.0000,Over
1,22:00,Eua Nba,Houston Rockets,Toronto Raptors,227.5,1.85,0.9989,Over
2,22:00,Eua Nba,Memphis Grizzlies,Golden State Warriors,223.5,1.83,0.9978,Over
3,16:00,Europa Euroliga,Crvena zvezda,Barcelona,162.5,1.91,0.8347,Over
4,16:30,Europa Euroliga,Monaco,Fenerbahce,161.5,1.80,1.0000,Over
